In [14]:
source_name=input('Enter Source name (Format="V1139_cyg"): ')

Enter Source name (Format="V1139_cyg"): NGC_2316


In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from sklearn.cluster import KMeans
from math import ceil

In [16]:
csv_path = fr"C:\Users\kgokh\Downloads\TAMOONS NEW FILE\selected_ysos_{source_name}.csv"
ysos_df = pd.read_csv(csv_path)

In [17]:
ra_center_deg = ysos_df['RA_deg'].median()
dec_center_deg = ysos_df['DEC_deg'].median()
coord_center = SkyCoord(ra=ra_center_deg*u.deg, dec=dec_center_deg*u.deg)
print(f" Using median center: RA={ra_center_deg}, DEC={dec_center_deg}")

 Using median center: RA=104.926663, DEC=-7.8474125


In [18]:
ysos_df['coord'] = [SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
                    for ra, dec in zip(ysos_df['RA_deg'], ysos_df['DEC_deg'])]
ysos_df['offset_arcmin'] = [c.separation(coord_center).arcminute for c in ysos_df['coord']]

In [19]:
ysos_df = ysos_df[(ysos_df['offset_arcmin'] > 0.3) & (ysos_df['offset_arcmin'] <= 6)]

In [20]:
ysos_df['Jmag_bin'] = ysos_df['Jmag'].apply(lambda x: round(x * 2) / 2)
magnitude_groups = ysos_df.groupby('Jmag_bin')

In [21]:
initial_sublists = []
for mag, group in magnitude_groups:
    group = group.sort_values('offset_arcmin')
    chunks = [] 
    i = 0        
    while i < len(group):
        chunk = group.iloc[i:i+8].copy()  
        chunks.append(chunk)              
        i += 8                            
    initial_sublists.extend(chunks)

In [22]:
def collision_check(sublist, min_sep=1.0):
    valid_targets = [] 
    for index, row in sublist.iterrows():
        initial_star = row  
        initial_star_coord = initial_star['coord']
        is_far_enough = True
        for selected_star in valid_targets:
            other_coord = selected_star['coord']
            separation = initial_star_coord.separation(other_coord).arcminute 
            if separation < min_sep:
                is_far_enough = False
                break 
        if is_far_enough:
            valid_targets.append(initial_star) 
    return pd.DataFrame(valid_targets)

In [23]:
collision_check_sublists = []
for sublist in initial_sublists:
    filtered = collision_check(sublist)
    collision_check_sublists.append(filtered)

In [25]:
final_sublists_after_kmeans = []
for group in collision_check_sublists:
    if len(group) > 8:
        coords_array = np.array([
            [c.ra.degree, c.dec.degree] for c in group['coord']
        ])
        k = ceil(len(group) / 8)
        kmeans = KMeans(n_clusters=k, random_state=42).fit(coords_array)
        group['cluster'] = kmeans.labels_
        for label in range(k):
            cluster_group = group[group['cluster'] == label].drop(columns='cluster')
            final_sublists_after_kmeans.append(cluster_group)
    else:
        final_sublists_after_kmeans.append(group)

In [26]:
output = []
for i, group in enumerate(final_sublists_after_kmeans, 1):
    for _, row in group.iterrows():
        coord = row['coord']
        ra_hms, dec_dms = coord.to_string('hmsdms').split()
        output.append({
            'Group': i,
            'GAIA_Source_ID': row['GAIA_Source_ID'],
            'RA_HMS': ra_hms,
            'DEC_DMS': dec_dms,
            'RA_deg': coord.ra.degree,
            'DEC_deg': coord.dec.degree,
            'Jmag': row['Jmag'],
            'Offset_arcmin': row['offset_arcmin']
        })

df_out = pd.DataFrame(output)
df_out.to_csv(f"grouped_ysos_{source_name}.csv", index=False)
print(f"Total groups: {len(final_sublists_after_kmeans)}. Results saved to 'grouped_ysos_{source_name}.csv'.")

Total groups: 12. Results saved to 'grouped_ysos_NGC_2316.csv'.
